In [1]:
%%writefile C:\Anaconda\Lib\WEIRDOFunctions.py

import datetime
import pandas as pd
import numpy as np

def ThetaCurve_mm_5pt(PSI,SAT,DUL,LL15,PSIbub):
    """This is the curve describing soil water content inrelation to soil water tension.  
    
    Args:
        PSI: An array of water potentials between -10 and -6e7 (Units mmH20)
        SAT: Water content at -10 mm H20 tension (Units mm^3/mm^3)
        DUL: Water content at -1000 mmH20 tension (Units mm^3/mm^3)
        LL15: Water content at -150000 mmH2O tension (Units mm^3/mm^3)
        BubPSI: Tension that air enters soil matrix (Units mmH20)
            
    Returns:
        An array of theta values corresponding to the PSI array input
    """
    ##Set up parameters for spline
    BUB = SAT
    PSIsat = -10
    PSIdul = -1000
    PSIll15 = -150000.0;
    PSIairdry = -0.6e8
    
    DELk = [0] * 5
    DELk[0] = (BUB - (SAT+1e-20)) / (np.log10(-PSIbub))#Tiny amount added to Sat so in situations where DUL = SAT this function returns a non zero value
    DELk[1] = (DUL- BUB) / (np.log10(-PSIdul) - np.log10(-PSIbub))
    DELk[2] = (LL15 - DUL) / (np.log10(-PSIll15) - np.log10(-PSIdul))
    DELk[3] = -LL15 / (np.log10(-PSIairdry) - np.log10(-PSIll15))
    DELk[4] = -LL15 / (np.log10(-PSIairdry) - np.log10(-PSIll15))
    
    Mk = [0] * 5
    Mk[0] = 0.0
    Mk[1] = (DELk[0] + DELk[1]) / 2.0
    Mk[2] = (DELk[1] + DELk[2]) / 2.0
    Mk[3] = (DELk[2] + DELk[3]) / 2.0
    Mk[4] = DELk[4]

    # First bit might not be monotonic so check and adjust
    alpha = Mk[0] / DELk[0]
    beta = Mk[1] / DELk[0]
    phi = alpha - (np.power(2.0 * alpha + beta - 3.0, 2.0) / (3.0 * (alpha + beta - 2.0)));
    if (phi <= 0):
        tau = 3.0 / np.sqrt(alpha * alpha + beta * beta)
        Mk[0] = tau * alpha * DELk[0]
        Mk[1] = tau * beta * DELk[0]

    M0 = [0] * 6
    M1 = [0] * 6
    Y0 = [0] * 6
    Y1 = [0] * 6

    M0[0] = 0.0;
    M1[0] = 0.0;
    Y0[0] = SAT;
    Y1[0] = SAT;
    
    M0[1] = Mk[0] * (np.log10(-PSIbub) - np.log10(-PSIsat));
    M1[1] = Mk[1] * (np.log10(-PSIbub) - np.log10(-PSIsat));
    Y0[1] = SAT;
    Y1[1] = BUB;

    M0[2] = Mk[1] * (np.log10(-PSIdul) - np.log10(-PSIbub));
    M1[2] = Mk[2] * (np.log10(-PSIdul) - np.log10(-PSIbub));
    Y0[2] = BUB;
    Y1[2] = DUL;

    M0[3] = Mk[2] * (np.log10(-PSIll15) - np.log10(-PSIdul));
    M1[3] = Mk[3] * (np.log10(-PSIll15) - np.log10(-PSIdul));
    Y0[3] = DUL;
    Y1[3] = LL15;

    M0[4] = Mk[3] * (np.log10(-PSIairdry) - np.log10(-PSIll15));
    M1[4] = Mk[4] * (np.log10(-PSIairdry) - np.log10(-PSIll15));
    Y0[4] = LL15;
    Y1[4] = 0.0;

    M0[5] = 0.0;
    M1[5] = 0.0;
    Y0[5] = 0.0;
    Y1[5] = 0.0;

    Thetas = []
    for psi in PSI:
        if (psi >= PSIsat):
            i = 0;
            t = 0.0;
        elif (psi > PSIbub):
            i = 1
            t = (np.log10(-psi) - np.log10(-PSIsat)) / (np.log10(-PSIbub) - np.log10(-PSIsat));
        elif (psi > PSIdul):
            i = 2;
            t = (np.log10(-psi) - np.log10(-PSIbub)) / (np.log10(-PSIdul) - np.log10(-PSIbub));
        elif (psi > PSIll15):
            i = 3;
            t = (np.log10(-psi) - np.log10(-PSIdul)) / (np.log10(-PSIll15) - np.log10(-PSIdul));
        elif (psi > PSIairdry):
            i = 4;
            t = (np.log10(-psi) - np.log10(-PSIll15)) / (np.log10(-PSIairdry) - np.log10(-PSIll15));
        else:
            i = 5;
            t = 0.0;

        tSqr = t * t;
        tCube = tSqr * t;
        theta = (2 * tCube - 3 * tSqr + 1) * Y0[i] + (tCube - 2 * tSqr + t) * M0[i]\
                + (-2 * tCube + 3 * tSqr) * Y1[i] + (tCube - tSqr) * M1[i];
        Thetas.append(min(theta, SAT))#When Sat and DUL are very close, spline can produce number greater that sat
    return Thetas

def ThetaCurve_mm_4pt(PSI, SAT,DUL,LL15):
    """This is the curve describing soil water content inrelation to soil water tension.  
    
    Args:
        PSI: An array of water potentials between -10 and -6e7 (Units mmH20)
        SAT: Water content at -10 mm H20 tension (Units mm^3/mm^3)
        DUL: Water content at -1000 mmH20 tension (Units mm^3/mm^3)
        LL15: Water content at -150000 mmH2O tension (Units mm^3/mm^3)
            
    Returns:
        An array of theta values corresponding to the PSI array input
    """
    ##Set up parameters for spline
    PSIsat = -10
    PSIdul = -1000
    PSIll15 = -150000.0;
    PSIairdry = -0.6e8
    
    DELk = [0] * 4
    DELk[0] = (DUL - (SAT+1e-20)) / (np.log10(-PSIdul))#Tiny amount added to Sat so in situations where DUL = SAT this function returns a non zero value
    DELk[1] = (LL15 - DUL) / (np.log10(-PSIll15) - np.log10(-PSIdul))
    DELk[2] = -LL15 / (np.log10(-PSIairdry) - np.log10(-PSIll15))
    DELk[3] = -LL15 / (np.log10(-PSIairdry) - np.log10(-PSIll15))

    Mk = [0] * 4
    Mk[0] = 0.0
    Mk[1] = (DELk[0] + DELk[1]) / 2.0
    Mk[2] = (DELk[1] + DELk[2]) / 2.0
    Mk[3] = DELk[3]

    # First bit might not be monotonic so check and adjust
    alpha = Mk[0] / DELk[0]
    beta = Mk[1] / DELk[0]
    phi = alpha - (np.power(2.0 * alpha + beta - 3.0, 2.0) / (3.0 * (alpha + beta - 2.0)));
    if (phi <= 0):
        tau = 3.0 / np.sqrt(alpha * alpha + beta * beta)
        Mk[0] = tau * alpha * DELk[0]
        Mk[1] = tau * beta * DELk[0]

    M0 = [0] * 5
    M1 = [0] * 5
    Y0 = [0] * 5
    Y1 = [0] * 5

    M0[0] = 0.0;
    M1[0] = 0.0;
    Y0[0] = SAT;
    Y1[0] = SAT;

    M0[1] = Mk[0] * (np.log10(-PSIdul) - np.log10(-PSIsat));
    M1[1] = Mk[1] * (np.log10(-PSIdul) - np.log10(-PSIsat));
    Y0[1] = SAT;
    Y1[1] = DUL;

    M0[2] = Mk[1] * (np.log10(-PSIll15) - np.log10(-PSIdul));
    M1[2] = Mk[2] * (np.log10(-PSIll15) - np.log10(-PSIdul));
    Y0[2] = DUL;
    Y1[2] = LL15;

    M0[3] = Mk[2] * (np.log10(-PSIairdry) - np.log10(-PSIll15));
    M1[3] = Mk[3] * (np.log10(-PSIairdry) - np.log10(-PSIll15));
    Y0[3] = LL15;
    Y1[3] = 0.0;

    M0[4] = 0.0;
    M1[4] = 0.0;
    Y0[4] = 0.0;
    Y1[4] = 0.0;

    Thetas = []
    Test = []
    for psi in PSI:
        if (psi >= PSIsat):
            i = 0;
            t = 0.0;
        elif (psi > PSIdul):
            i = 1;
            t = (np.log10(-psi) - np.log10(-PSIsat)) / (np.log10(-PSIdul) - np.log10(-PSIsat));
        elif (psi > PSIll15):
            i = 2;
            t = (np.log10(-psi) - np.log10(-PSIdul)) / (np.log10(-PSIll15) - np.log10(-PSIdul));
        elif (psi > PSIairdry):
            i = 3;
            t = (np.log10(-psi) - np.log10(-PSIll15)) / (np.log10(-PSIairdry) - np.log10(-PSIll15));
        else:
            i = 4;
            t = 0.0;

        tSqr = t * t;
        tCube = tSqr * t;
        theta = (2 * tCube - 3 * tSqr + 1) * Y0[i] + (tCube - 2 * tSqr + t) * M0[i]\
                + (-2 * tCube + 3 * tSqr) * Y1[i] + (tCube - tSqr) * M1[i];
        Thetas.append(min(theta, SAT))#When Sat and DUL are very close, spline can produce number greater that sat
    return Thetas

def PsiLower(x,FitFrame):
    if x == 10:
        ret_val = -6.000000e+07
    else:
        ret_val = FitFrame.ix[x+1,'PsiUpper']
    return ret_val

def LowerDia(x, FitFrame):
    if x == 10:
        ret_val = 0
    else:
        ret_val = FitFrame.ix[x+1,'UpperDiameter']
    return ret_val

def thetaLower(x, FitFrame):
    if x == 10:
        ret_val = 0
    else:
        ret_val = FitFrame.ix[x+1,'ThetaUpper']
    return ret_val

def AccumCond(x, FitFrame):
    if x == 10:
        ret_val = 0
    else:
        ret_val = FitFrame.ix[x,'PoreCapillarity'] + FitFrame.ix[x+1,'Conductivity']
    return ret_val

def ConductivityFrame(BoundPSI, FitSAT,FitDUL,FitLL,PSIbub,Cflow,Xflow):
    """Calculates and returns the components of the WEIRDO hydraulic conductivity model.
    Args:
        BoundPSI: An array of water potentials between -10 and -6e7 (Units mmH20)
        FitSAT: Water content at -10 mm H20 tension (Units mm^3/mm^3)
        FitDUL: Water content at -1000 mmH20 tension (Units mm^3/mm^3)
        FitLL: Water content at -150000 mmH2O tension (Units mm^3/mm^3)
        BubPSI: Tension that air enters soil matrix (Units mmH20)
        Cflow: Parameter for capillary conductivity
        Xflow: Parameter for capillary conductivity
            
    Returns:
        A Data frame containing the components of the hydraulic conductivity calculations and hydraulic conductivity
    """
    # PsiUpper units = mmH2O
    FitFrame = pd.DataFrame(data=BoundPSI,columns = ['PsiUpper'])
    # PsiUpper units = mmH2O
    FitFrame.ix[:,'PsiLower'] = [PsiLower(x, FitFrame) for x in range(FitFrame.index.size)]
     # LowerDiameter Units = um
    FitFrame.ix[:,'UpperDiameter'] = -30000/FitFrame.ix[:,'PsiUpper']
     # LowerDiameter Units = um
    FitFrame.ix[:,'LowerDiameter'] = 0
    FitFrame.ix[:,'LowerDiameter'] = [LowerDia(x, FitFrame) for x in range(FitFrame.index.size)]
    # Radius units = um
    FitFrame.ix[:,'PoreRadius'] = (FitFrame.ix[:,'UpperDiameter']+FitFrame.ix[:,'LowerDiameter'])/4
    # Area units = um2
    FitFrame.ix[:,'PoreArea'] = np.pi * np.power(FitFrame.ix[:,'PoreRadius'],2)
    # ThetaUpper Units = mm3/mm3
    #FitFrame.ix[:,'ThetaUpper'] = ThetaCurvemm5pt(FitSAT,FitDUL,FitLL,BoundPSI,psibub)
    FitFrame.ix[:,'ThetaUpper'] = ThetaCurve_mm_5pt(BoundPSI,FitSAT,FitDUL,FitLL,PSIbub)
    # ThetaLower Units = mm3/mm3
    FitFrame.ix[:,'ThetaLower'] = [thetaLower(x, FitFrame) for x in range(FitFrame.index.size)]
    # Volume Units = mm3/mm3
    FitFrame.ix[:,'Volume'] = FitFrame.ix[:,'ThetaUpper']-FitFrame.ix[:,'ThetaLower']
    # In cases where two point on the moisture release curve are close, the spline can
    # dip causing negative volumes.  Set these to zero
    FitFrame.ix[:,'Volume'].where(FitFrame.ix[:,'Volume']>0,0,inplace=True)
    # Number units = /m  Area is divided by 1e12 to convert from um2 to m2
    FitFrame.ix[:,'Number'] = np.divide(FitFrame.ix[:,'Volume'],np.divide(FitFrame.ix[:,'PoreArea'], 1e12))
    # PoreFlowRate units = mm3/s
    FitFrame.ix[:,'PoreFlowRate'] = Cflow * np.power(FitFrame.ix[:,'PoreRadius'],Xflow)
    # VolumetricFlowRate units = mm3/s/m2
    FitFrame.ix[:,'VolumetricFlow'] = FitFrame.ix[:,'PoreFlowRate'] * FitFrame.ix[:,'Number']
    # PoreCapilalarity units = mm/h  multiply by 3600 to go from seconds to hours and divide by 1e6 to go from mm3/m2 to mm depth of water
    FitFrame.ix[:,'PoreCapillarity'] = FitFrame.ix[:,'VolumetricFlow']/1e6*3600
    # Conductivity (of all pores) units = mm/h
    FitFrame.ix[:,'Conductivity'] = [0] * 11
    for x in reversed(range(FitFrame.index.size)):
        FitFrame.ix[x,'Conductivity'] = AccumCond(x, FitFrame)
    return FitFrame

def SorptionFrame(BoundPSI, FitSAT,FitDUL,FitLL,PSIbub,Cflow,Xflow):
    """Calculates and returns the components of the WEIRDO hydraulic conductivity model including sorption.
    Args:
        BoundPSI: An array of water potentials between -10 and -6e7 (Units mmH20)
        FitSAT: Water content at -10 mm H20 tension (Units mm^3/mm^3)
        FitDUL: Water content at -1000 mmH20 tension (Units mm^3/mm^3)
        FitLL: Water content at -150000 mmH2O tension (Units mm^3/mm^3)
        BubPSI: Tension that air enters soil matrix (Units mmH20)
        Cflow: Parameter for capillary conductivity
        Xflow: Parameter for capillary conductivity
            
    Returns:
        A Data frame containing the components of the hydraulic conductivity calculations and hydraulic conductivity
    """
    SorptionFrame = ConductivityFrame(BoundPSI, FitSAT,FitDUL,FitLL,PSIbub,Cflow,Xflow)
    SorptionFrame.ix[:,'Sorbtivity'] = np.sqrt(((7.4/SorptionFrame.ix[:,'PoreRadius']*1000) * SorptionFrame.ix[:,'PoreCapillarity'])/0.5)
    SorptionFrame.ix[:,'Sorption'] = 0.5 * SorptionFrame.ix[:,'Sorbtivity'] * np.power(1,-0.5)  
    SorptionFrame.ix[:,'Kin'] = SorptionFrame.ix[:,'PoreCapillarity'] + SorptionFrame.ix[:,'Sorption'] 
    SorptionFrame.ix[:,'CumCapil'] = 0
    for x in reversed(SorptionFrame.index):
        if x < 10:
            SorptionFrame.ix[x,'CumCapil'] = SorptionFrame.ix[x+1,'CumCapil'] + SorptionFrame.ix[x,'PoreCapillarity']
    SorptionFrame.ix[:,'CumSorp'] = 0
    for x in reversed(SorptionFrame.index):
        if x < 10:
            SorptionFrame.ix[x,'CumSorp'] = SorptionFrame.ix[x+1,'CumSorp'] + SorptionFrame.ix[x,'Sorbtivity']
    SorptionFrame.ix[:,'Cumkin'] = SorptionFrame.ix[:,'CumCapil'] + SorptionFrame.ix[:,'CumSorp']
    return SorptionFrame

Overwriting C:\Anaconda\Lib\WEIRDOFunctions.py
